### MINIMAL EXAMPLE OF JUPYTER NOTEBOOK WHICH CAN BE RUN WITH CROMWELL

### The problem is that it produces run2.html and trial_v1_movie_rec.gif in local folder

### IMPORT NECESSARY MODULES

In [ ]:
#pip install matplotlib
#!pip install pyro-ppl --upgrade

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import logging
import sys
from datetime import datetime
import moviepy.editor as mpy
import numpy as np
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1" #for debugging, it decrease performance dramatically

In [ ]:
%matplotlib inline  
#%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML, Image

In [ ]:
#!pip install pyro-ppl

In [ ]:
from utilities import *
from vae_model import * 

import torch
import pyro

# Set up pyro environment
pyro.clear_param_store()
pyro.set_rng_seed(0)

# Check versions
print("pyro.__version__  --> ",pyro.__version__)
print("torch.__version__ --> ",torch.__version__)
assert(pyro.__version__.startswith('1.2'))
assert(torch.__version__.startswith('1.4'))

## read two json files and combine in single dictionary

In [ ]:
params = load_json_as_dict("parameters.json")
print(params)

### prepare the file names

In [ ]:
# CROMWELL will localize: 
# gs://ld-data-bucket/data/fashionmnist_train.pkl -> execution_dir/ld-data-bucket/data/fashionmnist_train.pkl
# Therefore I just need to remove  "gs://"
# Note that every path is relative to the execution_dir

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text  # or whatever

train_file = remove_prefix(params["wdl.file_train"],"gs://")
test_file = remove_prefix(params["wdl.file_train"],"gs://")
ckpt_file = remove_prefix(params["wdl.file_ckpt"],"gs://")
dir_output = params["wdl.dir_output"]

# create output directionry if it does nto exists
try:
    os.mkdir(dir_output)
except FileExistsError:
    pass

# Save file to output dir
json_param_file = os.path.join(dir_output, "parameters.json")
save_dict_as_json(params,json_param_file)

# checks
assert os.path.isfile(train_file)
assert os.path.isfile(test_file)
if params["simulation"]["type"] != "scratch":
    assert os.path.isfile(ckpt_file)
    
print(train_file)
print(test_file)
print(dir_output)
print(json_param_file)

### get the data

In [ ]:
logging.info("get the data")
train_dataset = DatasetInMemory(train_file,use_cuda=torch.cuda.is_available())
test_dataset  = DatasetInMemory(test_file,use_cuda=torch.cuda.is_available())

In [ ]:
train_dataset.check()

In [ ]:
test_dataset.check()

### Instantiate model and optimizer

In [ ]:
vae = VaeClass(params)
optimizer = instantiate_optimizer(vae, params)

### There are 3 possible simulation types: scratch, resumed, pretrained

In [ ]:
logging.info("simulation type = "+str(params["simulation"]["type"]))

if (params["simulation"]["type"] == "scratch"):
    
    epoch_restart = -1
    history_dict = {}
    min_test_loss = 99999999

elif (params["simulation"]["type"] == "resumed"):
        
    resumed = load_info(path=params["simulation"]["path_to_file"], 
                        load_epoch=True, 
                        load_history=True)
    epoch_restart = resumed.epoch
    history_dict = resumed.history_dict
    min_test_loss = min(history_dict["test_loss"])
    
    load_model_optimizer(path=params["simulation"]["path_to_file"], 
                         model=vae,
                         optimizer=optimizer)

elif (params["simulation"]["type"] == "pretrained"):
       
    epoch_restart = -1
    history_dict = {}
    min_test_loss = 99999999
    
    load_model_optimizer(path=params["simulation"]["path_to_file"], 
                         model=vae,
                         optimizer=None)
    
# instantiate the scheduler if necessary    
if params["training"]["scheduler_is_active"]:
    scheduler = instantiate_scheduler(optimizer, params)

### Train loop

In [ ]:
TEST_FREQUENCY = params["training"]["TEST_FREQUENCY"]
CHECKPOINT_FREQUENCY = params["training"]["CHECKPOINT_FREQUENCY"]
NUM_EPOCHS = params["training"]["EPOCHS"]
BATCH_SIZE = params["training"]["batch_size"]

for delta_epoch in range(1,NUM_EPOCHS+1):
    epoch = delta_epoch+epoch_restart
    vae.train()   
    
    #with torch.autograd.set_detect_anomaly(True):
    with torch.autograd.set_detect_anomaly(False):
        train_metrics = train_one_epoch(vae, 
                                        train_dataset, 
                                        optimizer, 
                                        BATCH_SIZE, 
                                        verbose=(epoch==0), 
                                        weight_clipper=None)
        s = pretty_print_metrics(epoch, train_metrics, is_train=True)
        print(s)
            
        
        history_dict = add_named_tuple_to_dictionary(namedtuple=train_metrics, 
                                                     dictionary=history_dict,
                                                     key_prefix="train_")
        
    if params["training"]["scheduler_is_active"]:
        scheduler.step(epoch=epoch)
    
    if(epoch % TEST_FREQUENCY == 0):
        vae.eval()
        test_metrics = train_one_epoch(vae, 
                                       test_dataset, 
                                       optimizer, 
                                       BATCH_SIZE, 
                                       verbose=(epoch==0), 
                                       weight_clipper=None)
        
        s = pretty_print_metrics(epoch, test_metrics, is_train=False)
        print(s)
                
        history_dict = add_named_tuple_to_dictionary(namedtuple=test_metrics, 
                                                     dictionary=history_dict,
                                                     key_prefix="test_")
        
        test_loss = test_metrics["loss"]
        min_test_loss = min(min_test_loss, test_loss)
            
        #if((test_loss == min_test_loss) or ((epoch % CHECKPOINT_FREQUENCY) == 0)): 
        if((test_loss == min_test_loss) or ((epoch % TEST_FREQUENCY) == 0)):
            checkpoint_file = os.path.join(dir_output, "ckp_"+str(epoch)+".pkl")
            history_file = os.path.join(dir_output, "history_"+str(epoch)+".pkl")
            
            save_everything(model=vae, 
                            optimizer=optimizer, 
                            history_dict=history_dict, 
                            epoch=epoch, 
                            params_dict=params, 
                            path=checkpoint_file)
            
            save_dict_as_json(history_dict, path=history_file)
            print("saved files -> "+checkpoint_file+"  "+history_file)

# Check the results

In [ ]:
history_dict.keys()

In [ ]:
for k,v in history_dict.items():
    print(k," -->", history_dict[k][-3:])

In [ ]:
#plt.yscale('log')
y_shift=0
x_shift=0
sign=1

fontsize=10
fig, ax = plt.subplots(1,1)
ax.set_xlabel('REC',fontsize=fontsize)
ax.set_ylabel('REG',fontsize=fontsize)

ax.plot(np.arange(x_shift, x_shift+len(history_dict["train_loss"])), sign*np.array(history_dict["train_loss"])+y_shift,'-')
ax.plot(np.arange(x_shift, x_shift+len(history_dict["test_loss"])*TEST_FREQUENCY,TEST_FREQUENCY), sign*np.array(history_dict["test_loss"])+y_shift, '.--')
ax.set_xlabel('epoch')
ax.set_ylabel('LOSS = - ELBO')
ax.set_title('Training procedure')
ax.grid(True)
ax.legend(['train', 'test_clean', 'test_noisy'])

fig.tight_layout()
tmp_file = os.path.join(dir_output, "loss.png")
fig.savefig(tmp_file)

In [ ]:
# Plot of KL vs evidence
fontsize=20
labelsize=20

how_many = 2000
scale= 1
N = len(history_dict["train_kl"][-how_many :])
colors = np.arange(0.0,N,1.0)/N

fig, ax = plt.subplots(1,1)
#plt.yscale('log')
#plt.xlim(xmin=1.0, xmax=1.5)
ax.set_xlabel('REC',fontsize=fontsize)
ax.set_ylabel('REG',fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=labelsize)
ax.scatter(history_dict["train_nll"][-how_many :], history_dict["train_kl"][-how_many :],c=colors)
ax.plot(history_dict["train_nll"][-how_many :], history_dict["train_kl"][-how_many :], '-')
ax.grid()
#plt.xlim(xmax=2.5)

fig.tight_layout()
tmp_file = os.path.join(dir_output, "rec_kl_trajectory.png")
fig.savefig(tmp_file) 

### Check reconstruction

In [ ]:
tmp_list = [291, 413, 133, 148, 1,2,3,4,5,6,7,8,9]
reference_imgs, labels=test_dataset.load(batch_size=9, indices=tmp_list)
metric, inference = vae.reconstruct_img(reference_imgs)

reconstruction_file = os.path.join(dir_output, "imgs_reconstructed.png")
reference_file = os.path.join(dir_output, "imgs_reference.png")

imgs_ref = show_batch(reference_imgs[:],n_col=3,n_padding=4,title="REFERENCE")
imgs_ref.savefig(reference_file)

imgs_rec = show_batch(inference.reconstruction, n_col=3,n_padding=4, title="REC_IMG")
imgs_rec.savefig(reconstruction_file)

display(imgs_rec, imgs_ref)

# MAKE MOVIE

In [ ]:
epoch="xxx"
a = show_batch(inference.reconstruction[:9],n_col=3,n_padding=4,title="EPOCH = "+str(epoch))
display(a)

# actual loop

In [ ]:
rec_filenames = []

for epoch in range(0,30,TEST_FREQUENCY):
    if(epoch<10):
        label ="_000"+str(epoch)
    elif(epoch<100):
        label = "_00"+str(epoch)
    elif(epoch<1000):
        label = "_0"+str(epoch)
    elif(epoch<10000):
        label = "_"+str(epoch)
    else:
        raise Exception

    try:
        checkpoint_file = os.path.join(dir_output, "ckp_"+str(epoch)+".pkl")
        _ = load_model_optimizer(path=checkpoint_file, model=vae, optimizer=None)
        metric, inference = vae.reconstruct_img(reference_imgs)
        tmp_fig = show_batch(inference.reconstruction[:8],n_col=4,n_padding=4,title="EPOCH = "+str(epoch))
        tmp_rec_file = os.path.join(dir_output, "imgs_rec"+label+".png")
        rec_filenames.append(tmp_rec_file)
        tmp_fig.savefig(tmp_rec_file, bbox_inches='tight') 
    except:
        pass

print(rec_filenames)

## Check individual images

In [ ]:
def show_frame_rec(n):
    tmp = Image(filename=rec_filenames[n])
    return display(tmp)

def show_frame_all(n):
    c = Image(filename=rec_filenames[n])
    return display(c)

# make gif file

In [ ]:
movie_rec_file_local = "movie_rec.gif"
movie_rec_file_absolute = os.path.join(dir_output, movie_rec_file_local)

frame_per_second = 2
im = mpy.ImageSequenceClip(rec_filenames, fps=frame_per_second)
im.write_gif(movie_rec_file_local, fps=frame_per_second)
im.write_gif(movie_rec_file_absolute, fps=frame_per_second)

In [ ]:
HTML("<img src="+movie_rec_file_local+"></img>")

In [ ]:
show_frame_rec(0)

In [ ]:
show_batch(reference_imgs[:8],n_col=4,n_padding=4,title="REFERENCE")